In [1]:
import torch
from torch import nn

/opt/anaconda3/envs/legend13/lib/python3.12/site-packages/torch/_subclasses/functional_tensor.py:295: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_numpy.cpp:84.)
  cpu = _conversion_method_template(device=torch.device("cpu"))


In [2]:
cfgs = { "A": [64, "M", 128, "M", 256, 256, "M", 512, 512, "M", 512, 512, "M"],
        "B": [64, 64, "M", 128, 128, "M", 256, 256, "M", 512, 512, "M", 512, 512, "M"],
        "D": [64, 64, "M", 128, 128, "M", 256, 256, 256, "M", 512, 512, 512, "M", 512, 512, 512, "M"],
        "E": [64, 64, "M", 128, 128, "M", 256, 256, 256, 256, "M", 512, 512, 512, 512, "M", 512, 512, 512, 512, "M"]}

### VGGnet

In [3]:
class VGG(nn.Module):
    # VGG 초기화 Method
    def __init__(self, cfg, batch_norm, num_classes = 1000, init_weights = True, drop_p = 0.5):
        super().__init__()

        self.features = self.make_layers(cfg, batch_norm)
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7)) 
        self.classifier = nn.Sequential(nn.Linear(512 * 7 * 7, 4096),
                                        nn.ReLU(),
                                        nn.Dropout(p=drop_p),
                                        nn.Linear(4096, 4096),
                                        nn.ReLU(),
                                        nn.Dropout(p=drop_p),
                                        nn.Linear(4096, num_classes))

        if init_weights:
            for m in self.modules():
                if isinstance(m, nn.Conv2d):
                    nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity="relu")
                    if m.bias is not None:
                        nn.init.constant_(m.bias, 0)
                elif isinstance(m, nn.Linear):
                    nn.init.normal_(m.weight, 0, 0.01)
                    nn.init.constant_(m.bias, 0)

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

    def make_layers(self, cfg, batch_norm = False):
        layers = []
        in_channels = 3
        for v in cfg: # cfg = [64, 64, "M", 128, 128, "M", 256, 256, 256, "M", 512, 512, 512, "M", 512, 512, 512, "M"]
            if type(v) == int:
                if batch_norm:
                    layers += [nn.Conv2d(in_channels, v, 3, padding=1), # 어차피 BN에 bias 포함
                               nn.BatchNorm2d(v),
                               nn.ReLU()]
                else:
                    layers += [nn.Conv2d(in_channels, v, 3, padding=1),
                               nn.ReLU()]
                in_channels = v
            else:
                layers += [nn.MaxPool2d(2)]

        return nn.Sequential(*layers)

In [4]:
avgpool = nn.AdaptiveAvgPool2d((4, 4))
print(avgpool(torch.randn(2,3,32,32)).shape)
x = torch.randn(2,3,2,2)
print(avgpool(x)) # 작은 놈이 들어오면 늘려서라도 맞춰준다 # 값을 복제 시켜놓음

torch.Size([2, 3, 4, 4])
tensor([[[[ 0.9711,  0.9711,  2.6029,  2.6029],
          [ 0.9711,  0.9711,  2.6029,  2.6029],
          [-0.4300, -0.4300, -1.4447, -1.4447],
          [-0.4300, -0.4300, -1.4447, -1.4447]],

         [[-0.0884, -0.0884, -0.6124, -0.6124],
          [-0.0884, -0.0884, -0.6124, -0.6124],
          [-1.2006, -1.2006, -0.9367, -0.9367],
          [-1.2006, -1.2006, -0.9367, -0.9367]],

         [[ 1.6828,  1.6828,  1.3609,  1.3609],
          [ 1.6828,  1.6828,  1.3609,  1.3609],
          [ 0.1269,  0.1269, -0.0492, -0.0492],
          [ 0.1269,  0.1269, -0.0492, -0.0492]]],


        [[[ 0.5927,  0.5927,  1.1156,  1.1156],
          [ 0.5927,  0.5927,  1.1156,  1.1156],
          [-0.7794, -0.7794,  0.6039,  0.6039],
          [-0.7794, -0.7794,  0.6039,  0.6039]],

         [[ 0.4624,  0.4624, -0.5096, -0.5096],
          [ 0.4624,  0.4624, -0.5096, -0.5096],
          [-0.2471, -0.2471, -1.0650, -1.0650],
          [-0.2471, -0.2471, -1.0650, -1.0650]],

   

In [5]:
model = nn.Sequential(nn.Linear(2,2),
                      nn.ReLU(),
                      nn.Sequential(nn.Linear(2,3),
                                    nn.ReLU(),
                                    nn.Linear(3,3),
                                    nn.ReLU()),
                      nn.Linear(3,10))
[m for m in model.modules()]

[Sequential(
   (0): Linear(in_features=2, out_features=2, bias=True)
   (1): ReLU()
   (2): Sequential(
     (0): Linear(in_features=2, out_features=3, bias=True)
     (1): ReLU()
     (2): Linear(in_features=3, out_features=3, bias=True)
     (3): ReLU()
   )
   (3): Linear(in_features=3, out_features=10, bias=True)
 ),
 Linear(in_features=2, out_features=2, bias=True),
 ReLU(),
 Sequential(
   (0): Linear(in_features=2, out_features=3, bias=True)
   (1): ReLU()
   (2): Linear(in_features=3, out_features=3, bias=True)
   (3): ReLU()
 ),
 Linear(in_features=2, out_features=3, bias=True),
 ReLU(),
 Linear(in_features=3, out_features=3, bias=True),
 ReLU(),
 Linear(in_features=3, out_features=10, bias=True)]

In [6]:
# model1 = nn.Sequential([nn.Linear(1,1),
#                        nn.Linear(1,1)]) # 리스트를 넣으면 안돼요!

model2 = nn.Sequential(nn.Linear(1,1),
                       nn.Linear(1,1))

# print(*[1,2])
# print([1,2])

# model3 = nn.Sequential(nn.Linear(1,1),
#                         nn.Linear(1,1))

In [9]:
model = VGG(cfgs["E"], batch_norm=True)
# print(model)
from torchinfo import summary
summary(model, input_size=(2,3,224,224), device='cpu')

Layer (type:depth-idx)                   Output Shape              Param #
VGG                                      [2, 1000]                 --
├─Sequential: 1-1                        [2, 512, 7, 7]            --
│    └─Conv2d: 2-1                       [2, 64, 224, 224]         1,792
│    └─BatchNorm2d: 2-2                  [2, 64, 224, 224]         128
│    └─ReLU: 2-3                         [2, 64, 224, 224]         --
│    └─Conv2d: 2-4                       [2, 64, 224, 224]         36,928
│    └─BatchNorm2d: 2-5                  [2, 64, 224, 224]         128
│    └─ReLU: 2-6                         [2, 64, 224, 224]         --
│    └─MaxPool2d: 2-7                    [2, 64, 112, 112]         --
│    └─Conv2d: 2-8                       [2, 128, 112, 112]        73,856
│    └─BatchNorm2d: 2-9                  [2, 128, 112, 112]        256
│    └─ReLU: 2-10                        [2, 128, 112, 112]        --
│    └─Conv2d: 2-11                      [2, 128, 112, 112]        147,

In [10]:
x = torch.randn(2,3,224,224)
print(model(x).shape)

torch.Size([2, 1000])


In [11]:
x = torch.randn(2,3,300,300)
print(model(x).shape)

torch.Size([2, 1000])


In [12]:
x = torch.randn(2,3,32,32)
print(model(x).shape)

torch.Size([2, 1000])
